# 07 Data Wrangling
__Math 3080: Fundamentals of Data Science__

Reading:
* McKinney, Chapter 8 - Data Wrangling: Join, Combine, and Reshape

Outline:
1. Joining two datasets
2. Pivot tables
3. Groupbys